<a href="https://colab.research.google.com/github/sujin-create/Fintech_Project/blob/main/carddata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymysql
!pip install matplotlib
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd 
import io
import pymysql 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
sns.set()

In [53]:
from google.colab import files
myfile = files.upload()

Saving hyundaicard_20220813.csv to hyundaicard_20220813 (2).csv


In [54]:
df = pd.read_csv(io.BytesIO(myfile['hyundaicard_20220813.csv']))
df.head()

,date,storename,sector,price
0,2022-08-10,나이스 - 페이레터(주),쇼핑/유통,"1,000"
1,2022-08-09,KCP - 주식회사 위메프,쇼핑/유통,"2,900"
2,2022-08-01,토스페이먼츠주식회사 - ( 주 ) 야놀자,쇼핑/유통,"27,000"
3,2022-08-01,나이스 - 제이디스포츠패션코리아,쇼핑/유통,"20,500"
4,2022-07-19,지마켓_스마일카드 - 주식회사 지마켓,쇼핑/유통,"13,110"


In [63]:
df5 = df.drop(['date','storename'], axis=1)
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sector  185 non-null    object
 1   price   185 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


In [69]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sector  185 non-null    object
 1   price   185 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.0+ KB


In [70]:
print(df5.groupby(['sector'])['price'].agg('sum'))

sector
문화/오락/레져     576800
쇼핑/유통       1416654
요식           300700
의료(병/의원)     232400
Name: price, dtype: int64


In [ ]:
conn = pymysql.connect(host='autotrading-db.cjolqhecq70a.ap-northeast-2.rds.amazonaws.com',
                       user = 'admin',
                       password='autotrading1234',
                       db = 'Fintech',
                       charset = 'utf8')
curs = conn.cursor()

In [ ]:
tagged_corpus_list = []

for index, row in df.iterrows():
  text = row['sector']
  text = str(text)
  tag = row['storename']
  
  if index % 10 == 0:
    print(index,'번 수행했습니다.')

  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

0 번 수행했습니다.
10 번 수행했습니다.
20 번 수행했습니다.
30 번 수행했습니다.
40 번 수행했습니다.
50 번 수행했습니다.
60 번 수행했습니다.
70 번 수행했습니다.
80 번 수행했습니다.
90 번 수행했습니다.
100 번 수행했습니다.
110 번 수행했습니다.
120 번 수행했습니다.
130 번 수행했습니다.
140 번 수행했습니다.
150 번 수행했습니다.
160 번 수행했습니다.
170 번 수행했습니다.
180 번 수행했습니다.
190 번 수행했습니다.
200 번 수행했습니다.
210 번 수행했습니다.
220 번 수행했습니다.


In [ ]:
len(tagged_corpus_list)

223

In [ ]:
tagged_corpus_list[4]

TaggedDocument(words=['교통'], tags=['07월티머니지하철 14건'])

In [ ]:
from gensim.models import doc2vec
import time

In [ ]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)

print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

Tag Size: 130 / 

In [ ]:
tagged_corpus_list

[TaggedDocument(words=['쇼핑', '/', '유통'], tags=['나이스 - 페이레터(주)']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['KCP - 주식회사 위메프']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['토스페이먼츠주식회사 - ( 주 ) 야놀자']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['나이스 - 제이디스포츠패션코리아']),
 TaggedDocument(words=['교통'], tags=['07월티머니지하철 14건']),
 TaggedDocument(words=['교통'], tags=['07월티머니버스 3건']),
 TaggedDocument(words=['교통'], tags=['우티택시_KCP']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['지마켓_스마일카드 - 주식회사 지마켓']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['지마켓_스마일카드 - 주식회사 지마켓']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['토스페이먼츠주식회사 - (주)무신사']),
 TaggedDocument(words=['요식'], tags=['한국맥도날드외대점']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['GS25외대후레쉬점']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['토스페이먼츠주식회사 - (주)무신사']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['쮸쮸아이스크림이문점']),
 TaggedDocument(words=['쇼핑', '/', '유통'], tags=['씨유외대앞역점']),
 TaggedDocument(words=['쇼핑', '/', '유통

In [ ]:
%ls

'card_data (1).csv'   card_data.csv
'card_data (2).csv'   dart.doc2vec
'card_data (3).csv'  'hyundaicard_20220813 (1).csv'
'card_data (4).csv'   hyundaicard_20220813.csv
'card_data (5).csv'   images/
'card_data (6).csv'   install_mecab-ko_on_colab190912.sh
'card_data (7).csv'   install_mecab-ko_on_colab_light_220429.sh
'card_data (8).csv'   LICENSE
'card_data (9).csv'   README.md


In [ ]:
# 모델을 로드합니다.
model = doc2vec.Doc2Vec.load('dart.doc2vec')

In [ ]:
similar_doc = model.docvecs.most_similar(tagged_corpus_list[0].tags)
print(similar_doc)

[('백수약국', 0.9973117113113403), ('화양꽃집', 0.9971983432769775), ('미체로의원', 0.9971816539764404), ('더연세유외과의원', 0.997164249420166), ('오셀롯PC수원매탄직영점', 0.9971440434455872), ('한국외국어대학교후생과', 0.9971410036087036), ('신사역365약국', 0.9970935583114624), ('보물섬약국', 0.9970906376838684), ('브리즈피씨카페', 0.9970859885215759), ('리코스타코인노래연습장외대2호점', 0.9970569610595703)]


In [ ]:
import requests
import re
from bs4 import BeautifulSoup

#키워드 : 공인중개사
url = "https://search.naver.com/search.naver?where=view&sm=tab_jum&query=" + match
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
res = requests.get(url, headers=headers)
res.raise_for_status()
idx = 0

soup = BeautifulSoup(res.text, "lxml") # 가져온 문서를 lxml 파서를 통해서 BeautifulSoup객체로 만듬. soup에 모든 정보가 들어있음.

gongin = soup.find_all("div", attrs={"class":"total_wrap api_ani_send"})

for gongins in gongin:
    title = gongins.find("a", attrs={"class":"api_txt_lines total_tit _cross_trigger"}).get_text()
    link = gongins.a["href"]
    p = re.compile("cafe")
    m = p.search(link)

    if m:
        #print("카페 제외")
        continue
    print(title)
    print(link)
    idx = idx + 1

    if idx > 9 :
        break

폴로랄프로렌 자켓, 니트 추천 :] 폴로랄프로렌 현대백화점 목동점 방문 리뷰
#
현대백화점 목동점 폴로 랄프로렌 베어 니트 스웨터 내돈내산 후기샷
#
대구 현대백화점:) 폴로 랄프로렌 피마코튼 가디건
#
폴로 랄프로렌 건클럽체크 블레이져, 네이비 헤링본 블레이져, 브라운 헤링본 블레이져
#
현대백화점 폴로 랄프로렌 신상 셔츠원피스 선물 받았어요 꺅~
#
설 연휴 폴로 랄프로렌, 클럽모나코 득템 후기
#
폴로랄프로렌 베이스볼 캡 / 김포현대아울렛 폴로 모자네이비, 블랙 득템
#
220524 송도 현대 프리미엄 아울렛 폴로 랄프로렌 여름상품 둘러보기, 송현아
#
현대백화점 유아 폴로랄프로렌 니트 + 셔츠원피스
#
폴로 랄프로렌 현대백화점 12카드 적용되나요??
#
